## Hente rådata fra Frost-API-et vi har brukt.

Her i denne koden har vi hentet værdata fra Meteorologisk Institutt. Vi har valgt å hente data herfra fordi vi vet at Meteorologisk Institutt er et annerkjent norsk statlig forvaltningsorgan, med ansvar for den offentlige meteorologiske tjenesten i Norge. I og med at det er driftet av staten har det en viss integeritet og pålitelighet. 

Vi har fått en "client-id" som gir oss tilgang til å hente rådata fra Frost-API-et til Meteorologisk Institutt. I koden under henter vi da rådata mellom "start_dato" og "slutt_dato" og lagrer det i csv-filen "VaerData.csv" for visualisering. 

In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json

# Load environment variables from .env file
load_dotenv()

# Henter API-key/Client-id fra .env filen
client_id = os.getenv("client_id")

with open('../data/Byer.json', 'r') as f:
    byer = json.load(f)

byerAsString = ','.join(byer.values())

element_id = "mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)"  # Temperatur, nedbør og vindhastighet
start_dato = "2022-01-01"  # Startdato
slutt_dato = "2023-01-01"  # Sluttdato

# Define endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    "sources": byerAsString,
    "elements": element_id,
    "referencetime": f"{start_dato}/{slutt_dato}",
    "timeoffsets": "default",
    "timeresolutions": "P1D",
    "levels": "default"
}
print(f"Parametere: {parameters}")

# Sender forespørsel til Frost API
response = requests.get(endpoint, params=parameters, auth=(client_id, ''))

# Sjekker om forespørselen var vellykket
if response.status_code != 200:
    print(f"Error! Status code: {response.status_code}")
    print(f"Message: {response.json().get('error', {}).get('message', 'Ingen melding gitt')}")
    print(f"Reason: {response.json().get('error', {}).get('reason', 'Ingen årsak gitt')}")
    exit()

# Henter JSON-data
json_data = response.json()
if 'data' not in json_data:
    print("Ingen data funnet i forespørselen.")
    print(json_data)
    exit()

# Prosesserer data til en DataFrame
data = json_data['data']
rows = []  # Liste for å samle alle rader
for item in data:
    for observation in item['observations']:
        row = {
            'sourceId': item['sourceId'].split(':')[0],  # Fjern ':0' fra sourceId
            'referenceTime': item['referenceTime'],
            'elementId': observation['elementId'],
            'value': observation['value'],
            'unit': observation['unit'],
            'codequality': observation.get('qualityCode', '')  
        }
        rows.append(row)

# Opprett DataFrame fra rader
df = pd.DataFrame(rows)

# Legger til en ny kolonne for bynavn basert på sourceId
df['by'] = df['sourceId'].map({v: k for k, v in byer.items()})

# Forenkler tidsformatet (YYYY-MM-DD)
df['referenceTime'] = pd.to_datetime(df['referenceTime']).dt.date

# Splitter "elementId" i flere kolonner
df[['statistikk', 'variable']] = df['elementId'].str.extract(r'(\w+)\(([^)]+)')

# Velger ønskede kolonner
df = df[['by', 'sourceId', 'referenceTime', 'statistikk', 'variable', 'value', 'unit', 'codequality']]

# Skriver DataFrame til en CSV-fil
data_fil = '../data/VaerData.csv'
df.to_csv(data_fil, index=False)
print(f"Data lagret i {data_fil}")
print("Dataene er hentet fra Meterologisk Institutt.")

Parametere: {'sources': 'SN68230,SN18700,SN50540', 'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)', 'referencetime': '2022-01-01/2023-01-01', 'timeoffsets': 'default', 'timeresolutions': 'P1D', 'levels': 'default'}
Data lagret i ../data/VaerData.csv
Dataene er hentet fra Meterologisk Institutt.
